In [1]:
import os
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import io

In [12]:
# Full Drive access scope
SCOPES = ['https://www.googleapis.com/auth/drive']

# Authenticate and create the Drive service
def authenticate():
    creds = None
    if os.path.exists('../token.json'):
        creds = Credentials.from_authorized_user_file('../token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                '../credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('../token.json', 'w') as token:
            token.write(creds.to_json())
    return build('drive', 'v3', credentials=creds)

def upload_file(file_path, file_name):
    service = authenticate()
    file_metadata = {'name': file_name}
    media = MediaFileUpload(file_path, resumable=True)
    uploaded_file = service.files().create(
        body=file_metadata, media_body=media, fields='id').execute()
    print(f"File uploaded successfully. File ID: {uploaded_file.get('id')}")
    return uploaded_file.get('id')

# Example usage
file_id = upload_file('covertype.csv', 'covertype.csv')

File uploaded successfully. File ID: 1Vk5hDfLVwZ3MT-RXDr3XwUo2IoXP897a


In [13]:
def download_file(file_id, destination_path):
    service = authenticate()
    request = service.files().get_media(fileId=file_id)

    with io.FileIO(destination_path, 'wb') as file:
        downloader = MediaIoBaseDownload(file, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Download {int(status.progress() * 100)}%.")


# Example usage: Download the uploaded file
download_file("15YqRiq5ZBL6lk63baOnFDEvYRTwVGCtr", 'downloaded_myfile.csv')

Download 100%.


## IF TOKEN.JSON has expired then please generate new token by running the code below

In [10]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build


def get_credentials(*,
                    scopes: list = ['https://www.googleapis.com/auth/drive'],
                    token_path: str = '../token.json',
                    credentials_path: str = '../credentials.json'
                    ) -> Credentials:
    """Given a path to a saved token (which may not exist) and a path to
    your credentials file, return a `Credentials` instance.
    """

    def recertify():
        """Create a new Credentials instance using InstalledAppFlow."""
        flow = InstalledAppFlow.from_client_secrets_file(
            credentials_path, scopes)
        return flow.run_local_server(port=0)

    creds = None

    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists(token_path):
        # We have a token file. Recreate the credentials"
        creds = Credentials.from_authorized_user_file(token_path, scopes)
        if creds.valid:
            # We have valid credentials
            return creds

    # Either token_path does not exist or the credentials are no longer valid.
    if creds and creds.expired and creds.refresh_token:
        # The credentials have expired. Try to refresh the credentials:
        try:
            creds.refresh(Request())
        except Exception:
            # Probaly the refresh token has expired, so we must start anew
            creds = recertify()
    else:
        creds = recertify()

    # Save the credentials for the next run
    with open(token_path, 'w') as token_file:
        token_file.write(creds.to_json())

    return creds


def main():
    creds = get_credentials()  # Take defaults
    service = build('drive', 'v3', credentials=creds)

In [11]:
main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=357494762777-d8c7n5h169p4bvu974ponipsejdrvt05.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A54312%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=0DoM8heDpJRBsLrLgUUdab58q6877S&access_type=offline
